In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [3]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


We see CustomerID is not req as we already have rowno and id wont create any impact on our target variable . Also surname is also not req.

In [11]:
x = df.iloc[:,3:-1]
y = df.iloc[:,-1]


Splitted data into dependent and independent variable x  and Y


---



In [12]:
x

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [13]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In x we see we have categorical variable geography and gender . hence
we need to encode it

In [14]:
x = pd.get_dummies(x,columns = ['Geography','Gender'])

In [15]:
x

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,0,0,1


As we have Geography_France	Geography_Germany we can drop Geography_Spain as any one of then will 1 rest will be 0. Same for male and female.

In [18]:
x.drop(['Geography_Spain','Gender_Female'],axis =1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,1


In [19]:
df.corr() #only age has correlation of 0.28 with exited

<ipython-input-19-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.006495,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
sc = StandardScaler()

In [22]:
x_stand = sc.fit_transform(x)
x_stand

array([[-0.32622142,  0.29351742, -1.04175968, ..., -0.57380915,
         1.09598752, -1.09598752],
       [-0.44003595,  0.19816383, -1.38753759, ...,  1.74273971,
         1.09598752, -1.09598752],
       [-1.53679418,  0.29351742,  1.03290776, ..., -0.57380915,
         1.09598752, -1.09598752],
       ...,
       [ 0.60498839, -0.27860412,  0.68712986, ..., -0.57380915,
         1.09598752, -1.09598752],
       [ 1.25683526,  0.29351742, -0.69598177, ..., -0.57380915,
        -0.91241915,  0.91241915],
       [ 1.46377078, -1.04143285, -0.35020386, ..., -0.57380915,
         1.09598752, -1.09598752]])

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x_stand,y,test_size = 0.25,random_state = 7)

Building DNN

In [31]:
dnn = tf.keras.models.Sequential()
#adding first hidden layer
dnn.add(tf.keras.layers.Dense(units = 6,activation="relu"))
#adding second hidden layer
dnn.add(tf.keras.layers.Dense(units = 6,activation="relu"))
#output
dnn.add(tf.keras.layers.Dense(units = 1,activation="sigmoid"))
#training the model
dnn.compile(optimizer = "adam",loss = "binary_crossentropy",metrics = ["accuracy"])



In [32]:
#training the dataset and fitting the model
dnn.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
235/235 [==============================] - 7s 3ms/step - loss: 0.6299 - accuracy: 0.6875
Epoch 2/100
235/235 [==============================] - 1s 3ms/step - loss: 0.5025 - accuracy: 0.7964
Epoch 3/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4572 - accuracy: 0.7979
Epoch 4/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4319 - accuracy: 0.8137
Epoch 5/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4157 - accuracy: 0.8243
Epoch 6/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4041 - accuracy: 0.8317
Epoch 7/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3943 - accuracy: 0.8413
Epoch 8/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3860 - accuracy: 0.8457
Epoch 9/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3778 - accuracy: 0.8500
Epoch 10/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3699 - accura

In [44]:
dnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 6)                 84        
                                                                 
 dense_10 (Dense)            (None, 6)                 42        
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 133 (532.00 Byte)
Trainable params: 133 (532.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Therefore we got loss value = 0.32 and accuracy = 0.8651
Agenda : Basics and forward and backward prapogation use
now if you see after certain epochs loss value an accuracy is not changing . this is called convergence

In [33]:
y_pred = dnn.predict(x_test)

79/79 [==============================] - 0s 1ms/step


In [36]:
y_pred = (y_pred>0.5)

In [34]:
from sklearn.metrics import accuracy_score,classification_report

In [42]:
print("Accuracy_score=",accuracy_score(y_test,y_pred))

Accuracy_score= 0.86


In [43]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1990
           1       0.75      0.47      0.58       510

    accuracy                           0.86      2500
   macro avg       0.81      0.71      0.75      2500
weighted avg       0.85      0.86      0.85      2500



acc score of train is 86.55 and of test is 86 .hence no bias no variance. Brilliant model